<a href="https://colab.research.google.com/github/soumyadweep/Learning-Data-Placeholders-for-Open-Set-Recognition/blob/main/resnet_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import time
import torch.nn as nn
import torchvision.models as models
from torchvision.transforms import ToPILImage, Grayscale

In [ ]:
class CustomMNISTDataset(torch.utils.data.Dataset):
    def __init__(self, root, transform = None, train=True):
        self.mnist_data = datasets.MNIST(root=root, train=train, transform=transform, download=True)
        self.train = train
        self.resize_transform = transforms.Resize((224, 224))

        if train:
            self.filtered_indices = [idx for idx, label in enumerate(self.mnist_data.targets) if label < 6]
        else:
            self.filtered_indices = [6 if label >= 6 else label for _, label in self.mnist_data]
        #self.transform = transform
        #self.to_pil = ToPILImage()
        #self.grayscale = Grayscale(num_output_channels=3)

    def __len__(self):
        return len(self.filtered_indices)

    def __getitem__(self, idx):
        if self.train:
            filtered_idx = self.filtered_indices[idx]
            image, label = self.mnist_data[filtered_idx]
        else:
            image, _ = self.mnist_data[idx]
            label = self.filtered_indices[idx]



        return image, label


In [ ]:
# Define the transformations
#transform = transforms.Compose([transforms.ToTensor()])
transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Lambda(lambda x: torch.cat([x, x, x], 0)),
                #transforms.Normalize(mean, std),
                 ])
## train_test dataset
train_dataset = CustomMNISTDataset(root='path_to_mnist_dataset12', transform=transform, train=True)
test_dataset = CustomMNISTDataset(root='path_to_mnist_dataset12', transform=transform, train=False)


## Train_Testest Dataloader
train_data_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_data_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [ ]:
# import torchvision.models as models
# resnet18 = models.resnet18(pretrained=True)

In [ ]:
 #import torch.nn as nn
 #resnet18.info()


In [ ]:
class myModel(nn.Module):
  def __init__(self):
    super(myModel,self).__init__()
    self.feat_extractor = models.resnet18(weights = 'IMAGENET1K_V1')
    self.feat_extractor.fc = nn.Identity()
    self.classifier = nn.Linear(512,7)
  def forward(self,img,label):
    feat = self.feat_extractor(img)
    ####

    ####
    prob = self.classifier(feat)
    return prob

In [ ]:
from torch.optim import optimizer
import tempfile
import os

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    # Create a temporary directory to save training checkpoints
    with tempfile.TemporaryDirectory() as tempdir:
        best_model_params_path = os.path.join(tempdir, 'best_model_params.pt')

        torch.save(model.state_dict(), best_model_params_path)
        best_acc = 0.0

        for epoch in range(num_epochs):
            print(f'Epoch {epoch}/{num_epochs - 1}')
            print('-' * 10)

            # Each epoch has a training and validation phase
            for phase in ['train', 'val']:
                if phase == 'train':
                    model.train()  # Set model to training mode
                else:
                    model.eval()   # Set model to evaluate mode

                running_loss = 0.0
                running_corrects = 0

                # Iterate over data.
                for inputs, labels in dataloaders[phase]:
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # forward
                    # track history if only in train
                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(inputs,labels)
                        _, preds = torch.max(outputs, 1)
                        loss = criterion(outputs, labels)

                        # backward + optimize only if in training phase
                        if phase == 'train':
                            loss.backward()
                            optimizer.step()

                    # statistics
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)
                #if phase == 'train':
                   # scheduler.step()

                epoch_loss = running_loss / (dataset_sizes[phase]*batch_size)
                epoch_acc = running_corrects.double() / (dataset_sizes[phase]*batch_size)

                print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

                # deep copy the model
                if phase == 'val' and epoch_acc > best_acc:
                    best_acc = epoch_acc
                    torch.save(model.state_dict(), best_model_params_path)

            print()

        time_elapsed = time.time() - since
        print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
        print(f'Best val Acc: {best_acc:4f}')

        # load best model weights
        model.load_state_dict(torch.load(best_model_params_path))
    return model

In [ ]:
from torch.optim import optimizer
import tempfile
import os
model = myModel().to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
dataloaders = {
    'train': train_data_loader,
    'val': test_data_loader
}
dataset_sizes = {
    'train': len(train_data_loader),
    'val':len(test_data_loader)
}

batch_size = 32

In [ ]:
trainresult = train_model(model, criterion, optimizer, scheduler, num_epochs=10)

Epoch 0/9
----------
train Loss: 0.0997 Acc: 0.9686
val Loss: 2.8458 Acc: 0.5991

Epoch 1/9
----------
train Loss: 0.0222 Acc: 0.9935
val Loss: 3.0185 Acc: 0.5977

Epoch 2/9
----------
train Loss: 0.0209 Acc: 0.9942
val Loss: 3.0029 Acc: 0.6003

Epoch 3/9
----------
train Loss: 0.0159 Acc: 0.9953
val Loss: 3.1236 Acc: 0.6009

Epoch 4/9
----------
train Loss: 0.0119 Acc: 0.9963
val Loss: 3.2548 Acc: 0.6011

Epoch 5/9
----------
train Loss: 0.0093 Acc: 0.9967
val Loss: 3.5846 Acc: 0.5997

Epoch 6/9
----------
train Loss: 0.0093 Acc: 0.9969
val Loss: 3.7193 Acc: 0.6001

Epoch 7/9
----------
train Loss: 0.0137 Acc: 0.9965
val Loss: 3.5659 Acc: 0.5996

Epoch 8/9
----------
train Loss: 0.0072 Acc: 0.9975
val Loss: 3.8394 Acc: 0.6009

Epoch 9/9
----------
train Loss: 0.0049 Acc: 0.9983
val Loss: 4.1165 Acc: 0.5998

Training complete in 4m 15s
Best val Acc: 0.601138
